In [18]:
import spotipy
import sys
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util

cid ="" 
secret = ""

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [178]:
# timeit library to measure the time needed to run this code
import timeit
start = timeit.default_timer()

# create empty lists where the results are going to be stored
artist_name = []
track_name = []
popularity = []
track_id = []

for i in range(0,2000,50):
    track_results = sp.search(q='year:2020', type='track', limit=50,offset=i)
    for i, t in enumerate(track_results['tracks']['items']):
        artist_name.append(t['artists'][0]['name'])
        track_name.append(t['name'])
        track_id.append(t['id'])
        popularity.append(t['popularity'])
      

stop = timeit.default_timer()
print ('Time to run this code (in seconds):', stop - start)


Time to run this code (in seconds): 8.90958099999989


In [179]:
print('number of elements in the track_id list:', len(track_id))

number of elements in the track_id list: 2000


In [180]:
import pandas as pd

df_tracks2020 = pd.DataFrame({'artist_name':artist_name,'track_name':track_name,'track_id':track_id,'popularity':popularity})
print(df_tracks2020.shape)
df_tracks2020.head(10)

(2000, 4)


,artist_name,track_name,track_id,popularity
0,Lil Mosey,Blueberry Faygo,6wJYhPfqk3KGhHRG76WzOh,95
1,Drake,Toosie Slide,127QTOFJsJQp5LbJbu3A1y,98
2,Future,Life Is Good (feat. Drake),5yY9lUy8nbvjM1Uyo1Uqoc,94
3,Powfu,death bed (coffee for your head) (feat. beabad...,7eJMfftS33KTjuF7lTsMCx,98
4,Jack Harlow,WHATS POPPIN,1jaTQ3nqY3oAAYyCTbIvnM,92
5,Eminem,Godzilla (feat. Juice WRLD),7FIWs0pqAYbP91WWM0vlTQ,94
6,Megan Thee Stallion,Savage,55CHeLEfn5iJ0IIkgaa4si,93
7,Lil Uzi Vert,P2,3kW4z2pIj5VY5fLjBrMRL5,88
8,Lil Uzi Vert,Myron,56uXDJRCuoS7abX3SkzHKQ,87
9,NLE Choppa,Walk Em Down (feat. Roddy Ricch),4cSSL3YafYjM3yjgFO1vJg,89


In [181]:
# group the entries by artist_name and track_name and check for duplicates
grouped = df_tracks2020.groupby(['artist_name','track_name'], as_index=True).size()
grouped[grouped > 1].count()

186

In [182]:
df_tracks2020.drop_duplicates(subset=['artist_name','track_name'], inplace=True)

In [183]:
grouped_after_dropping = df_tracks2020.groupby(['artist_name','track_name'], as_index=True).size()
grouped_after_dropping[grouped_after_dropping > 1].count()

0

In [184]:
df_tracks2020[df_tracks2020.duplicated(subset=['artist_name','track_name'],keep=False)].count()

artist_name    0
track_name     0
track_id       0
popularity     0
dtype: int64

In [185]:
df_tracks2020.shape

(1758, 4)

In [186]:
# again measuring the time
start = timeit.default_timer()

# empty list, batchsize and the counter for None results
rows = []
batchsize = 100
None_counter = 0

for i in range(0,len(df_tracks2020['track_id']),batchsize):
    batch = df_tracks2020['track_id'][i:i+batchsize]
    feature_results = sp.audio_features(batch)
    for i, t in enumerate(feature_results):
        if t == None:
            None_counter = None_counter + 1
        else:
            rows.append(t)
            
print('Number of tracks where no audio features were available:',None_counter)

stop = timeit.default_timer()
print ('Time to run this code (in seconds):',stop - start)

Number of tracks where no audio features were available: 0
Time to run this code (in seconds): 1.8501360999998724


In [187]:
print('number of elements in the track_id list:', len(rows))

number of elements in the track_id list: 1758


In [188]:
df_audio_features = pd.DataFrame.from_dict(rows,orient='columns')
print("Shape of the dataset:", df_audio_features.shape)
df_audio_features.head()

Shape of the dataset: (1758, 18)


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.774,0.554,0,-7.909,1,0.0383,0.2070,0.000000,0.132,0.349,99.034,audio_features,6wJYhPfqk3KGhHRG76WzOh,spotify:track:6wJYhPfqk3KGhHRG76WzOh,https://api.spotify.com/v1/tracks/6wJYhPfqk3KG...,https://api.spotify.com/v1/audio-analysis/6wJY...,162547,4
1,0.834,0.454,1,-9.750,0,0.2010,0.3210,0.000006,0.114,0.837,81.618,audio_features,127QTOFJsJQp5LbJbu3A1y,spotify:track:127QTOFJsJQp5LbJbu3A1y,https://api.spotify.com/v1/tracks/127QTOFJsJQp...,https://api.spotify.com/v1/audio-analysis/127Q...,247059,4
2,0.676,0.609,2,-5.831,0,0.4810,0.0706,0.000000,0.152,0.508,142.037,audio_features,5yY9lUy8nbvjM1Uyo1Uqoc,spotify:track:5yY9lUy8nbvjM1Uyo1Uqoc,https://api.spotify.com/v1/tracks/5yY9lUy8nbvj...,https://api.spotify.com/v1/audio-analysis/5yY9...,237735,4
3,0.726,0.431,8,-8.765,0,0.1350,0.7310,0.000000,0.696,0.348,144.026,audio_features,7eJMfftS33KTjuF7lTsMCx,spotify:track:7eJMfftS33KTjuF7lTsMCx,https://api.spotify.com/v1/tracks/7eJMfftS33KT...,https://api.spotify.com/v1/audio-analysis/7eJM...,173333,4
4,0.923,0.604,11,-6.671,0,0.2450,0.0170,0.000000,0.272,0.826,145.062,audio_features,1jaTQ3nqY3oAAYyCTbIvnM,spotify:track:1jaTQ3nqY3oAAYyCTbIvnM,https://api.spotify.com/v1/tracks/1jaTQ3nqY3oA...,https://api.spotify.com/v1/audio-analysis/1jaT...,139741,4


In [189]:
df_audio_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1758 entries, 0 to 1757
Data columns (total 18 columns):
danceability        1758 non-null float64
energy              1758 non-null float64
key                 1758 non-null int64
loudness            1758 non-null float64
mode                1758 non-null int64
speechiness         1758 non-null float64
acousticness        1758 non-null float64
instrumentalness    1758 non-null float64
liveness            1758 non-null float64
valence             1758 non-null float64
tempo               1758 non-null float64
type                1758 non-null object
id                  1758 non-null object
uri                 1758 non-null object
track_href          1758 non-null object
analysis_url        1758 non-null object
duration_ms         1758 non-null int64
time_signature      1758 non-null int64
dtypes: float64(9), int64(4), object(5)
memory usage: 247.3+ KB


In [190]:
columns_to_drop = ['analysis_url','track_href','type','uri']
df_audio_features.drop(columns_to_drop, axis=1,inplace=True)

df_audio_features.rename(columns={'id': 'track_id'}, inplace=True)

df_audio_features.shape


(1758, 14)

In [191]:
# merge both dataframes
# the 'inner' method will make sure that we only keep track IDs present in both datasets
df_tracks2020 = pd.merge(df_tracks2020,df_audio_features,on='track_id',how='inner')
print("Shape of the dataset:", df_audio_features.shape)
df_tracks2020.head()

Shape of the dataset: (1758, 14)


,artist_name,track_name,track_id,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Lil Mosey,Blueberry Faygo,6wJYhPfqk3KGhHRG76WzOh,95,0.774,0.554,0,-7.909,1,0.0383,0.2070,0.000000,0.132,0.349,99.034,162547,4
1,Drake,Toosie Slide,127QTOFJsJQp5LbJbu3A1y,98,0.834,0.454,1,-9.750,0,0.2010,0.3210,0.000006,0.114,0.837,81.618,247059,4
2,Future,Life Is Good (feat. Drake),5yY9lUy8nbvjM1Uyo1Uqoc,94,0.676,0.609,2,-5.831,0,0.4810,0.0706,0.000000,0.152,0.508,142.037,237735,4
3,Powfu,death bed (coffee for your head) (feat. beabad...,7eJMfftS33KTjuF7lTsMCx,98,0.726,0.431,8,-8.765,0,0.1350,0.7310,0.000000,0.696,0.348,144.026,173333,4
4,Jack Harlow,WHATS POPPIN,1jaTQ3nqY3oAAYyCTbIvnM,92,0.923,0.604,11,-6.671,0,0.2450,0.0170,0.000000,0.272,0.826,145.062,139741,4


In [192]:
df_tracks2020['YEAR']=2020

In [193]:
df_tracks2020.head()

,artist_name,track_name,track_id,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,YEAR
0,Lil Mosey,Blueberry Faygo,6wJYhPfqk3KGhHRG76WzOh,95,0.774,0.554,0,-7.909,1,0.0383,0.2070,0.000000,0.132,0.349,99.034,162547,4,2020
1,Drake,Toosie Slide,127QTOFJsJQp5LbJbu3A1y,98,0.834,0.454,1,-9.750,0,0.2010,0.3210,0.000006,0.114,0.837,81.618,247059,4,2020
2,Future,Life Is Good (feat. Drake),5yY9lUy8nbvjM1Uyo1Uqoc,94,0.676,0.609,2,-5.831,0,0.4810,0.0706,0.000000,0.152,0.508,142.037,237735,4,2020
3,Powfu,death bed (coffee for your head) (feat. beabad...,7eJMfftS33KTjuF7lTsMCx,98,0.726,0.431,8,-8.765,0,0.1350,0.7310,0.000000,0.696,0.348,144.026,173333,4,2020
4,Jack Harlow,WHATS POPPIN,1jaTQ3nqY3oAAYyCTbIvnM,92,0.923,0.604,11,-6.671,0,0.2450,0.0170,0.000000,0.272,0.826,145.062,139741,4,2020


In [391]:
data=pd.concat([df_tracks2010, df_tracks2011,df_tracks2012,df_tracks2013,df_tracks2014,df_tracks2015,df_tracks2016,df_tracks2017,df_tracks2018,df_tracks2019,df_tracks2020], ignore_index=True)

NameError: name 'df_tracks2010' is not defined

In [390]:
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21073 entries, 0 to 21072
Data columns (total 20 columns):
Unnamed: 0          21073 non-null int64
Unnamed: 0.1        21073 non-null int64
artist_name         21073 non-null object
track_name          21073 non-null object
popularity          21073 non-null int64
danceability        21073 non-null float64
energy              21073 non-null float64
key                 21073 non-null int64
loudness            21073 non-null float64
mode                21073 non-null int64
speechiness         21073 non-null float64
acousticness        21073 non-null float64
instrumentalness    21073 non-null float64
liveness            21073 non-null float64
valence             21073 non-null float64
tempo               21073 non-null float64
duration_ms         21073 non-null int64
time_signature      21073 non-null int64
YEAR                21073 non-null int64
mode2               21073 non-null object
dtypes: float64(9), int64(8), object(3)
memory usa